# Title
[]()

In [62]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# from plotly.subplots import make_subplots
import requests
import json
import re
from processing import *

In [38]:
datetime.timestamp(datetime.strptime('2023-02-01', "%Y-%m-%d"))

1675238400.0

In [39]:
datetime.strptime('2023-02-01', "%Y-%m-%d") < datetime.strptime('2023-02-21', "%Y-%m-%d")

True

In [40]:
with open("credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['ig_user_id']
access_token = credentials['access_token']

# `get_ig_account_insights`

In [47]:
from datetime import time, datetime, timedelta
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
            # print(f'Date range must be 30 days or less;')
            # print(f'"until" parameter changed to {datetime.strftime(since, "%Y-%m-%d")}')
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter >= since) & (page <10):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        page +=1
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        except:
            print('No data in request response for page', page)

        # try:
        #     next_endpoint = response_json_dict[page]['paging']['previous']
        #     if next_endpoint+access_token != url:
        #         url = next_endpoint
        #     else:
        #         print('end')
        #         break
        # except: 
        #     break
    try:
        df = pd.concat(df_list)
        print('Number of posts:',len(df))
    except:
        df = response
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df, response_json_dict = get_ig_account_insights(ig_user_id, access_token, since='2023-01-30', until='2023-02-24')

https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677225600.0&since=1675065600.0
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Requesting page 3...
	Response status code:  200
Requesting page 4...
	Response status code:  200
Requesting page 5...
	Response status code:  200
Requesting page 6...
	Response status code:  200
Requesting page 7...
	Response status code:  200
Requesting page 8...
	Response status code:  200
Requesting page 9...
	Response status code:  200
Number of posts: 18


In [54]:
since='2023-01-30'
since = datetime.strptime(since, "%Y-%m-%d")
since

datetime.datetime(2023, 1, 30, 0, 0)

In [68]:
print(response_json_dict[2]['data'][0]['values'][0]['end_time'])
timestamp = response_json_dict[2]['data'][0]['values'][0]['end_time']
# datetime.strptime(re.sub(r'(.+)T.+', r'\1', timestamp), "%Y-%m-%d")
datetime.fromtimestamp(timestamp)
# re.sub(r'(.+)T.+', r'\1', timestamp)

2023-01-30T08:00:00+0000


TypeError: an integer is required (got type str)

In [69]:
print(datetime.now())

2023-02-26 21:40:38.784413


In [52]:
response_json_dict[2]['data']

[{'name': 'impressions',
  'period': 'day',
  'values': [{'value': 196, 'end_time': '2023-01-30T08:00:00+0000'},
   {'value': 45, 'end_time': '2023-01-31T08:00:00+0000'},
   {'value': 14, 'end_time': '2023-02-01T08:00:00+0000'},
   {'value': 5, 'end_time': '2023-02-02T08:00:00+0000'},
   {'value': 169, 'end_time': '2023-02-03T08:00:00+0000'},
   {'value': 68, 'end_time': '2023-02-04T08:00:00+0000'},
   {'value': 57, 'end_time': '2023-02-05T08:00:00+0000'},
   {'value': 78, 'end_time': '2023-02-06T08:00:00+0000'},
   {'value': 266, 'end_time': '2023-02-07T08:00:00+0000'},
   {'value': 77, 'end_time': '2023-02-08T08:00:00+0000'},
   {'value': 21, 'end_time': '2023-02-09T08:00:00+0000'},
   {'value': 3, 'end_time': '2023-02-10T08:00:00+0000'},
   {'value': 1, 'end_time': '2023-02-11T08:00:00+0000'},
   {'value': 184, 'end_time': '2023-02-12T08:00:00+0000'},
   {'value': 48, 'end_time': '2023-02-13T08:00:00+0000'},
   {'value': 200, 'end_time': '2023-02-14T08:00:00+0000'},
   {'value': 40,

In [71]:
df.head()

,name,period,values,title,description,id
0,impressions,day,"[{'value': 196, 'end_time': '2023-01-30T08:00:...",Impressions,Total number of times the Business Account's m...,17841403231458630/insights/impressions/day
1,reach,day,"[{'value': 141, 'end_time': '2023-01-30T08:00:...",Reach,Total number of times the Business Account's m...,17841403231458630/insights/reach/day
0,impressions,day,"[{'value': 196, 'end_time': '2023-01-30T08:00:...",Impressions,Total number of times the Business Account's m...,17841403231458630/insights/impressions/day
1,reach,day,"[{'value': 141, 'end_time': '2023-01-30T08:00:...",Reach,Total number of times the Business Account's m...,17841403231458630/insights/reach/day
0,impressions,day,"[{'value': 196, 'end_time': '2023-01-30T08:00:...",Impressions,Total number of times the Business Account's m...,17841403231458630/insights/impressions/day


In [81]:
response_json_dict[2]['data'][0]

{'name': 'impressions',
 'period': 'day',
 'values': [{'value': 196, 'end_time': '2023-01-30T08:00:00+0000'},
  {'value': 45, 'end_time': '2023-01-31T08:00:00+0000'},
  {'value': 14, 'end_time': '2023-02-01T08:00:00+0000'},
  {'value': 5, 'end_time': '2023-02-02T08:00:00+0000'},
  {'value': 169, 'end_time': '2023-02-03T08:00:00+0000'},
  {'value': 68, 'end_time': '2023-02-04T08:00:00+0000'},
  {'value': 57, 'end_time': '2023-02-05T08:00:00+0000'},
  {'value': 78, 'end_time': '2023-02-06T08:00:00+0000'},
  {'value': 266, 'end_time': '2023-02-07T08:00:00+0000'},
  {'value': 77, 'end_time': '2023-02-08T08:00:00+0000'},
  {'value': 21, 'end_time': '2023-02-09T08:00:00+0000'},
  {'value': 3, 'end_time': '2023-02-10T08:00:00+0000'},
  {'value': 1, 'end_time': '2023-02-11T08:00:00+0000'},
  {'value': 184, 'end_time': '2023-02-12T08:00:00+0000'},
  {'value': 48, 'end_time': '2023-02-13T08:00:00+0000'},
  {'value': 200, 'end_time': '2023-02-14T08:00:00+0000'},
  {'value': 40, 'end_time': '2023-

In [82]:
response_json_dict[2]['data'][1]

{'name': 'reach',
 'period': 'day',
 'values': [{'value': 141, 'end_time': '2023-01-30T08:00:00+0000'},
  {'value': 43, 'end_time': '2023-01-31T08:00:00+0000'},
  {'value': 9, 'end_time': '2023-02-01T08:00:00+0000'},
  {'value': 4, 'end_time': '2023-02-02T08:00:00+0000'},
  {'value': 109, 'end_time': '2023-02-03T08:00:00+0000'},
  {'value': 51, 'end_time': '2023-02-04T08:00:00+0000'},
  {'value': 29, 'end_time': '2023-02-05T08:00:00+0000'},
  {'value': 67, 'end_time': '2023-02-06T08:00:00+0000'},
  {'value': 208, 'end_time': '2023-02-07T08:00:00+0000'},
  {'value': 52, 'end_time': '2023-02-08T08:00:00+0000'},
  {'value': 21, 'end_time': '2023-02-09T08:00:00+0000'},
  {'value': 3, 'end_time': '2023-02-10T08:00:00+0000'},
  {'value': 1, 'end_time': '2023-02-11T08:00:00+0000'},
  {'value': 98, 'end_time': '2023-02-12T08:00:00+0000'},
  {'value': 39, 'end_time': '2023-02-13T08:00:00+0000'},
  {'value': 131, 'end_time': '2023-02-14T08:00:00+0000'},
  {'value': 21, 'end_time': '2023-02-15T08

In [90]:
# json_normalize(response_json_dict[2], record_path='values', meta='data')
pd.concat([json_normalize(response_json_dict[2]['data'][0], record_path='values', record_prefix='impressions_'), json_normalize(response_json_dict[2]['data'][1], record_path='values')],
          axis=1)

,impressions_value,impressions_end_time,value,end_time
0,196,2023-01-30T08:00:00+0000,141,2023-01-30T08:00:00+0000
1,45,2023-01-31T08:00:00+0000,43,2023-01-31T08:00:00+0000
2,14,2023-02-01T08:00:00+0000,9,2023-02-01T08:00:00+0000
3,5,2023-02-02T08:00:00+0000,4,2023-02-02T08:00:00+0000
4,169,2023-02-03T08:00:00+0000,109,2023-02-03T08:00:00+0000
5,68,2023-02-04T08:00:00+0000,51,2023-02-04T08:00:00+0000
6,57,2023-02-05T08:00:00+0000,29,2023-02-05T08:00:00+0000
7,78,2023-02-06T08:00:00+0000,67,2023-02-06T08:00:00+0000
8,266,2023-02-07T08:00:00+0000,208,2023-02-07T08:00:00+0000
9,77,2023-02-08T08:00:00+0000,52,2023-02-08T08:00:00+0000


## version 2

In [96]:
from datetime import time, datetime, timedelta
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter >= since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        print('Number of days of data:',len(df))
    except:
        df = response
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df2, response_json_dict2 = get_ig_account_insights(ig_user_id, access_token, since='2022-06-30', until='2023-02-24')

https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677225600.0&since=1674633600.0
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Requesting page 3...
	Response status code:  200
Requesting page 4...
	Response status code:  200
Requesting page 5...
	Response status code:  200
Requesting page 6...
	Response status code:  200
Requesting page 7...
	Response status code:  200
Requesting page 8...
	Response status code:  200
Requesting page 9...
	Response status code:  200
Number of days of data: 270


In [97]:
df2

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
1,2,2023-01-26T08:00:00+0000,2,2023-01-26T08:00:00+0000
2,38,2023-01-27T08:00:00+0000,1,2023-01-27T08:00:00+0000
3,29,2023-01-28T08:00:00+0000,1,2023-01-28T08:00:00+0000
4,5,2023-01-29T08:00:00+0000,4,2023-01-29T08:00:00+0000
...,...,...,...,...
25,69,2022-06-25T07:00:00+0000,58,2022-06-25T07:00:00+0000
26,22,2022-06-26T07:00:00+0000,22,2022-06-26T07:00:00+0000
27,14,2022-06-27T07:00:00+0000,14,2022-06-27T07:00:00+0000
28,26,2022-06-28T07:00:00+0000,5,2022-06-28T07:00:00+0000


In [98]:
response_json_dict2

{1: {'data': [{'name': 'impressions',
    'period': 'day',
    'values': [{'value': 0, 'end_time': '2023-01-25T08:00:00+0000'},
     {'value': 2, 'end_time': '2023-01-26T08:00:00+0000'},
     {'value': 38, 'end_time': '2023-01-27T08:00:00+0000'},
     {'value': 29, 'end_time': '2023-01-28T08:00:00+0000'},
     {'value': 5, 'end_time': '2023-01-29T08:00:00+0000'},
     {'value': 196, 'end_time': '2023-01-30T08:00:00+0000'},
     {'value': 45, 'end_time': '2023-01-31T08:00:00+0000'},
     {'value': 14, 'end_time': '2023-02-01T08:00:00+0000'},
     {'value': 5, 'end_time': '2023-02-02T08:00:00+0000'},
     {'value': 169, 'end_time': '2023-02-03T08:00:00+0000'},
     {'value': 68, 'end_time': '2023-02-04T08:00:00+0000'},
     {'value': 57, 'end_time': '2023-02-05T08:00:00+0000'},
     {'value': 78, 'end_time': '2023-02-06T08:00:00+0000'},
     {'value': 266, 'end_time': '2023-02-07T08:00:00+0000'},
     {'value': 77, 'end_time': '2023-02-08T08:00:00+0000'},
     {'value': 21, 'end_time': '

# version 3

In [99]:
from datetime import time, datetime, timedelta
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter >= since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = response
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df3, response_json_dict3 = get_ig_account_insights(ig_user_id, access_token, 
    since='2022-11-30', until='2023-02-24', filename='my_IG_account_insights_2022-11-30_2023-02-24')

https://graph.facebook.com/v15.0/17841403231458630/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1677225600.0&since=1674633600.0
Requesting page 1...
	Response status code:  200
Requesting page 2...
	Response status code:  200
Requesting page 3...
	Response status code:  200
Number of days of data: 91
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/interim/my_IG_account_insights_2022-11-30_2023-02-24.csv
Time completed: 2023-02-26 22:15:03.271953
File saved:  C:/Users/silvh/OneDrive/lighthouse/portfolio-projects/online-PT-social-media-NLP/data/raw/my_IG_account_insights_2022-11-30_2023-02-24.sav
Time completed: 2023-02-26 22:15:03.280052


In [100]:
df3

,impressions_value,impressions_end_time,reach_value,reach_end_time
0,0,2023-01-25T08:00:00+0000,0,2023-01-25T08:00:00+0000
1,2,2023-01-26T08:00:00+0000,2,2023-01-26T08:00:00+0000
2,38,2023-01-27T08:00:00+0000,1,2023-01-27T08:00:00+0000
3,29,2023-01-28T08:00:00+0000,1,2023-01-28T08:00:00+0000
4,5,2023-01-29T08:00:00+0000,4,2023-01-29T08:00:00+0000
...,...,...,...,...
86,177,2022-12-21T08:00:00+0000,150,2022-12-21T08:00:00+0000
87,234,2022-12-22T08:00:00+0000,144,2022-12-22T08:00:00+0000
88,69,2022-12-23T08:00:00+0000,50,2022-12-23T08:00:00+0000
89,45,2022-12-24T08:00:00+0000,21,2022-12-24T08:00:00+0000


# process_account_insights

In [138]:
def process_df_timestamp2(input_df, timestamp_column='timestamp'):
    """
    Convert dates in the json-derived dataframe from Facebook API read requests
    into different formats.

    Parameters: 
        - input_df : DataFrame with the timestamp of the data.
        - timestamp_column (str): Name of the column with the timestamp.
    """
    df = input_df.reset_index(drop=True)
    regex_date = r'.+T'
    df[timestamp_column] = pd.to_datetime(df[timestamp_column])
    df['date'] = df[timestamp_column].dt.date
    df['year'] = df[timestamp_column].dt.year
    df['month'] = df[timestamp_column].dt.month
    df['week_of_year'] = df[timestamp_column].dt.isocalendar().week
    df['day_of_week'] = df[timestamp_column].dt.dayofweek
    df['time'] = df[timestamp_column].dt.time
    df['hour'] = df[timestamp_column].dt.hour
    return df

In [183]:

def process_account_insights(
    input_df, data_column_suffix='value', timestamp_column_suffix='end_time',
    streamlit=False, timezone='Canada/Pacific'
    ):

    timestamp_columns = input_df.filter(regex=timestamp_column_suffix).columns
    # Check to see if all the timestamp columns are identical before proceeding
    for index, column in enumerate(timestamp_columns[:-1]): 
        if input_df[column].equals(input_df[timestamp_columns[index+1]]):
            continue
        else:
            print('Error: timestamp columns are not all identical')
            break
    df = pd.concat([
        input_df[column], 
        input_df.filter(regex=data_column_suffix)
        ], axis=1)
    if timezone:
        print('Time zone:', timezone)
        df[column] = pd.to_datetime(df[column]).dt.tz_convert(timezone)
    else:
        print('Time stamps in UTC time')
    df.columns = df.columns.str.replace(f'_{data_column_suffix}', f'', regex=False)
    timezone_suffix = '_'+timezone if timezone else 'UTC'
    timestamp_column_name = f'metric_{timestamp_column_suffix}_{timezone_suffix}'
    df = df.rename(columns={column: timestamp_column_name})
    df = process_df_timestamp2(df, timestamp_column=timestamp_column_name) ## !! update the function
    
    return df

process_account_insights(df3)

Time zone: Canada/Pacific


,metric_end_time__Canada/Pacific,impressions,reach,date,year,month,week_of_year,day_of_week,time,hour
0,2023-01-25 00:00:00-08:00,0,0,2023-01-25,2023,1,4,2,00:00:00,0
1,2023-01-26 00:00:00-08:00,2,2,2023-01-26,2023,1,4,3,00:00:00,0
2,2023-01-27 00:00:00-08:00,38,1,2023-01-27,2023,1,4,4,00:00:00,0
3,2023-01-28 00:00:00-08:00,29,1,2023-01-28,2023,1,4,5,00:00:00,0
4,2023-01-29 00:00:00-08:00,5,4,2023-01-29,2023,1,4,6,00:00:00,0
...,...,...,...,...,...,...,...,...,...,...
86,2022-12-21 00:00:00-08:00,177,150,2022-12-21,2022,12,51,2,00:00:00,0
87,2022-12-22 00:00:00-08:00,234,144,2022-12-22,2022,12,51,3,00:00:00,0
88,2022-12-23 00:00:00-08:00,69,50,2022-12-23,2022,12,51,4,00:00:00,0
89,2022-12-24 00:00:00-08:00,45,21,2022-12-24,2022,12,51,5,00:00:00,0


In [184]:
process_account_insights(df3, timezone=None)

Time stamps in UTC time


,metric_end_time_UTC,impressions,reach,date,year,month,week_of_year,day_of_week,time,hour
0,2023-01-25 08:00:00+00:00,0,0,2023-01-25,2023,1,4,2,08:00:00,8
1,2023-01-26 08:00:00+00:00,2,2,2023-01-26,2023,1,4,3,08:00:00,8
2,2023-01-27 08:00:00+00:00,38,1,2023-01-27,2023,1,4,4,08:00:00,8
3,2023-01-28 08:00:00+00:00,29,1,2023-01-28,2023,1,4,5,08:00:00,8
4,2023-01-29 08:00:00+00:00,5,4,2023-01-29,2023,1,4,6,08:00:00,8
...,...,...,...,...,...,...,...,...,...,...
86,2022-12-21 08:00:00+00:00,177,150,2022-12-21,2022,12,51,2,08:00:00,8
87,2022-12-22 08:00:00+00:00,234,144,2022-12-22,2022,12,51,3,08:00:00,8
88,2022-12-23 08:00:00+00:00,69,50,2022-12-23,2022,12,51,4,08:00:00,8
89,2022-12-24 08:00:00+00:00,45,21,2022-12-24,2022,12,51,5,08:00:00,8


In [185]:
process_account_insights(df3, timezone='Australia/Sydney')

Time zone: Australia/Sydney


,metric_end_time__Australia/Sydney,impressions,reach,date,year,month,week_of_year,day_of_week,time,hour
0,2023-01-25 19:00:00+11:00,0,0,2023-01-25,2023,1,4,2,19:00:00,19
1,2023-01-26 19:00:00+11:00,2,2,2023-01-26,2023,1,4,3,19:00:00,19
2,2023-01-27 19:00:00+11:00,38,1,2023-01-27,2023,1,4,4,19:00:00,19
3,2023-01-28 19:00:00+11:00,29,1,2023-01-28,2023,1,4,5,19:00:00,19
4,2023-01-29 19:00:00+11:00,5,4,2023-01-29,2023,1,4,6,19:00:00,19
...,...,...,...,...,...,...,...,...,...,...
86,2022-12-21 19:00:00+11:00,177,150,2022-12-21,2022,12,51,2,19:00:00,19
87,2022-12-22 19:00:00+11:00,234,144,2022-12-22,2022,12,51,3,19:00:00,19
88,2022-12-23 19:00:00+11:00,69,50,2022-12-23,2022,12,51,4,19:00:00,19
89,2022-12-24 19:00:00+11:00,45,21,2022-12-24,2022,12,51,5,19:00:00,19


# *End of Page*